In [28]:
#imports
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
from keras import optimizers

In [29]:
DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = True

if DEV:
  epochs = 12
else:
  epochs = 50

train_data_path = './data/train'
validation_data_path = './data/validation'

"""
Parameters
"""
img_width, img_height = 150, 150
batch_size = 64
samples_per_epoch = 5400
validation_steps = 2100
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 8
lr = 0.0004

In [31]:
model = Sequential()
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Conv2D(nb_filters2, (conv2_size, conv2_size), padding ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_first'))

model.add(Flatten())
model.add(Dense(1024)) # increased dense model from 256 to 512
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'],)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 9311 images belonging to 8 classes.
Found 9311 images belonging to 8 classes.


In [32]:
model.fit_generator(
    train_generator,
    steps_per_epoch=samples_per_epoch // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps // batch_size)

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/modelv4.0_8.categories.h5')
model.save_weights('./models/weights_v4.0_8.categories.0.h5')

Epoch 1/50
84/84 [==============================] - 481s 6s/step - loss: 2.7375 - acc: 0.3220 - val_loss: 1.5712 - val_acc: 0.4648
Epoch 2/50
84/84 [==============================] - 464s 6s/step - loss: 1.6061 - acc: 0.4213 - val_loss: 1.4821 - val_acc: 0.4419
Epoch 3/50
84/84 [==============================] - 466s 6s/step - loss: 1.5209 - acc: 0.4600 - val_loss: 1.3238 - val_acc: 0.5483
Epoch 4/50
84/84 [==============================] - 463s 6s/step - loss: 1.4752 - acc: 0.4868 - val_loss: 1.4577 - val_acc: 0.4634
Epoch 5/50
84/84 [==============================] - 464s 6s/step - loss: 1.4335 - acc: 0.4961 - val_loss: 1.2457 - val_acc: 0.5645
Epoch 6/50
84/84 [==============================] - 461s 5s/step - loss: 1.3804 - acc: 0.5225 - val_loss: 1.2643 - val_acc: 0.5698
Epoch 7/50
84/84 [==============================] - 479s 6s/step - loss: 1.3624 - acc: 0.5286 - val_loss: 1.1662 - val_acc: 0.5938
Epoch 8/50
84/84 [==============================] - 464s 6s/step - loss: 1.3275 - a

In [34]:
#predict_multiclass
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model


img_width, img_height = 150, 150
model_path = './models/modelv4.0_8.categories.h5'
model_weights_path = './models/weights_v4.0_8.categories.0.h5'
model = load_model(model_path)
model.load_weights(model_weights_path)

def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("Label: banana")
  elif answer == 1:
    print("Labels: battery")
  elif answer == 2:
    print("Label: computer")
  elif answer == 3:
    print("Label: glass_bottel")
  elif answer == 4:
    print("Label: light_blub")
  elif answer == 5:
    print("Label: paper")
  elif answer == 6:
    print("Label: phone")
  elif answer == 7:
    print("Label: plastic")
  

  return answer

In [35]:
# classify a picture
for i, ret in enumerate(os.walk('./test-data/whatishere')):
  for i, filename in enumerate(ret[2]):
    result = predict(ret[0] + '/' + filename)

Label: phone
